In [1]:
import numpy as np
import pandas as pd
import scipy.stats as st
import matplotlib.pyplot as plt

In [39]:
jmax = 409 
imax = 5460 
S0 = 4577.11
Barr = 0.7*S0
sigma = 0.3
r = 0.1
q = 0
T = 546/365
SL = 0 
SU = 2.5*S0 
Face=1000
CPN=0.005375*Face

In [143]:
def CNFD(S0,Face,CPN, Barr, T, r, q, sigma, SU, jmaxmin, jmaxmax, jmaxstep, imax):
    cnfdb_result = []
    for jmax in range(jmaxmin, jmaxmax, jmaxstep):
        V = np.zeros([imax+1, jmax+1])
        VT=np.zeros([imax+1, jmax+1])
        A = np.zeros([jmax+1])
        B = np.zeros([jmax+1])
        C = np.zeros([jmax+1])
        D = np.zeros([jmax+1])
        alpha = np.zeros([jmax+1])
        CN_S = np.zeros([jmax+1])
        dt = T / imax
        dS = SU/jmax
        j0=int(S0/dS)
        if (j0*dS < S0): j0 = j0+1
        jb = int(Barr/dS)
        if (jb*dS > Barr): jb = jb-1
        ireview = [90,120,154,181,212,244,273,304,335,365,399,426,455,485,518,546]
        ireview = [round(i/365/T*imax) for i in ireview]
        ipayment=[37,64 ,93,125,157,184,217,247,276,309,338,370,402,429,458,490,521,549]
        ipayment = [round(i/365/T*imax) for i in ipayment]
        i = imax
        for j in range(0, jmax+1):
            if j*dS >= S0:
                VT[i, j] =(Face+CPN)*np.exp(-r*(ipayment[-1]*dt-ireview[-1]*dt))
            else:
                VT[i,j] = (Face*j*dS/S0+CPN)*np.exp(-r*(ipayment[-1]*dt-ireview[-1]*dt))
        for i in range(imax-1, -1, -1):
            if i in ireview:
                A[0] = 0
                B[0] = 1
                C[0] = 0
                D[0] = 0
                for x in (ireview):
                    if i <= x:
                        D[0]=D[0]+ CPN*np.exp(-r*(x*dt-i*dt))
                for j in range(0, j0, 1):
                    A[j] = 0.25*sigma**2*j**2 - 0.25*(r-q)*j
                    B[j] = -1/dt-0.5*r-0.5*sigma**2*j**2
                    C[j] = 0.25*sigma**2*j**2 + 0.25*(r-q)*j
                    D1 = -VT[i+1,j-1]*(0.25*sigma**2*j**2-0.25*(r-q)*j)
                    D2 = -VT[i+1,j]*(1/dt-0.5*r-0.5*sigma**2*j**2)
                    D3 = -VT[i+1,j+1]*(0.25*sigma**2*j**2+0.25*(r-q)*j)
                    D[j] = D1+D2+D3
                A[j0] = 0
                B[j0] = 1
                C[j0] = 0
                D[j0] = (Face+CPN)*np.exp(-r*(ipayment[ireview.index(i)+2]*dt-i*dt))
                alpha[0] = B[0]
                CN_S[0] = D[0]
                for j in range(1, j0+1, 1):
                    alpha[j] = B[j]-(A[j]*C[j-1])/alpha[j-1]
                    CN_S[j] = D[j]-(A[j]*CN_S[j-1])/alpha[j-1]  
                VT[i,j0] = CN_S[j0]/alpha[j0]
                for j in range(j0-1,-1,-1):
                    VT[i,j] = (CN_S[j]-C[j]*VT[i,j+1])/alpha[j]
                    VT[i,j]=VT[i,j]+CPN*np.exp(-r*(ipayment[ireview.index(i)+2]*dt-i*dt))
                for j in range(j0, jmax+1, 1):
                    VT[i, j] =(Face+CPN)*np.exp(-r*(ipayment[ireview.index(i)+2]*dt-i*dt))
            else:
                A[0] = 0
                B[0] = 1
                C[0] = 0
                D[0] = 0
                for x in (ireview):
                    if x >= i:
                        D[0]=D[0]+ CPN*np.exp(-r*(x*dt-i*dt))
                for j in range(0, jmax, 1):
                    A[j] = 0.25*sigma**2*j**2 - 0.25*(r-q)*j
                    B[j] = -1/dt-0.5*r-0.5*sigma**2*j**2
                    C[j] = 0.25*sigma**2*j**2 + 0.25*(r-q)*j
                    D1 = -VT[i+1,j-1]*(0.25*sigma**2*j**2-0.25*(r-q)*j)
                    D2 = -VT[i+1,j]*(1/dt-0.5*r-0.5*sigma**2*j**2)
                    D3 = -VT[i+1,j+1]*(0.25*sigma**2*j**2+0.25*(r-q)*j)
                    D[j] = D1+D2+D3
                A[jmax] = 0
                B[jmax] = 1
                C[jmax] = 0
                for x in range(len(ireview)-1):
                    if ireview[x]<i and ireview[x+1]>i:
                        D[jmax] = (Face+CPN)*np.exp(-r*(ipayment[ireview.index(ireview[x+1])+2]*dt-i*dt))
                if i < ireview[0]:
                    D[jmax] = (Face+CPN)*np.exp(-r*(ipayment[ireview.index(ireview[0])+2]*dt-i*dt))
                alpha[0] = B[0]
                CN_S[0] = D[0]
                for j in range(1, jmax+1, 1):
                    alpha[j] = B[j]-(A[j]*C[j-1])/alpha[j-1]
                    CN_S[j] = D[j]-(A[j]*CN_S[j-1])/alpha[j-1]  
                VT[i,jmax] = CN_S[jmax]/alpha[jmax]
                for j in range(jmax-1,-1,-1):
                    VT[i,j] = (CN_S[j]-C[j]*VT[i,j+1])/alpha[j]
            if i in ipayment[0:2]:
                VT[i,j]=VT[i,j]+CPN
                
                
        i=imax       
        for j in range(0, jmax+1):
            if j*dS >= Barr:
                V[i, j] =(Face+CPN)*np.exp(-r*(ipayment[-1]*dt-ireview[-1]*dt))
            else:
                V[i,j] = (Face*j*dS/S0+CPN)*np.exp(-r*(ipayment[-1]*dt-ireview[-1]*dt)) 
        for i in range(imax-1, -1, -1):
            for j in range(0, jb+1, 1):
                V[i,j]=VT[i,j]
            if i in ireview:
                A[jb] = 0
                B[jb] = 1
                C[jb] = 0
                D[jb] = VT[i,jb]
                for j in range(jb, j0, 1):
                    A[j] = 0.25*sigma**2*j**2 - 0.25*(r-q)*j
                    B[j] = -1/dt-0.5*r-0.5*sigma**2*j**2
                    C[j] = 0.25*sigma**2*j**2 + 0.25*(r-q)*j
                    D1 = -V[i+1,j-1]*(0.25*sigma**2*j**2-0.25*(r-q)*j)
                    D2 = -V[i+1,j]*(1/dt-0.5*r-0.5*sigma**2*j**2)
                    D3 = -V[i+1,j+1]*(0.25*sigma**2*j**2+0.25*(r-q)*j)
                    D[j] = D1+D2+D3
                A[j0] = 0
                B[j0] = 1
                C[j0] = 0
                D[j0] = (Face+CPN)*np.exp(-r*(ipayment[ireview.index(i)+2]*dt-i*dt))
                alpha[jb] = B[jb]
                CN_S[jb] = D[jb]
                for j in range(jb, j0+1, 1):
                    alpha[j] = B[j]-(A[j]*C[j-1])/alpha[j-1]
                    CN_S[j] = D[j]-(A[j]*CN_S[j-1])/alpha[j-1]  
                V[i,j0] = CN_S[j0]/alpha[j0]
                for j in range(j0-1,jb,-1):
                    V[i,j] = (CN_S[j]-C[j]*V[i,j+1])/alpha[j]
                    V[i,j]=V[i,j]+CPN*np.exp(-r*(ipayment[ireview.index(i)+2]*dt-i*dt))
                for j in range(j0, jmax+1, 1):
                    V[i, j] =(Face+CPN)*np.exp(-r*(ipayment[ireview.index(i)+2]*dt-i*dt))
            else:
                A[jb] = 0
                B[jb] = 1
                C[jb] = 0
                D[jb] = VT[i,jb]
                for j in range(jb, jmax, 1):
                    A[j] = 0.25*sigma**2*j**2 - 0.25*(r-q)*j
                    B[j] = -1/dt-0.5*r-0.5*sigma**2*j**2
                    C[j] = 0.25*sigma**2*j**2 + 0.25*(r-q)*j
                    D1 = -V[i+1,j-1]*(0.25*sigma**2*j**2-0.25*(r-q)*j)
                    D2 = -V[i+1,j]*(1/dt-0.5*r-0.5*sigma**2*j**2)
                    D3 = -V[i+1,j+1]*(0.25*sigma**2*j**2+0.25*(r-q)*j)
                    D[j] = D1+D2+D3
                A[jmax] = 0
                B[jmax] = 1
                C[jmax] = 0
                for x in range(len(ireview)-1):
                    if ireview[x]<i and ireview[x+1]>i:
                        D[jmax] = (Face+CPN)*np.exp(-r*(ipayment[ireview.index(ireview[x+1])+2]*dt-i*dt))
                if i < ireview[0]:
                    D[jmax] = (Face+CPN)*np.exp(-r*(ipayment[ireview.index(ireview[0])+2]*dt-i*dt))
                alpha[jb] = B[jb]
                CN_S[jb] = D[jb]
                for j in range(jb+1, jmax+1, 1):
                    alpha[j] = B[j]-(A[j]*C[j-1])/alpha[j-1]
                    CN_S[j] = D[j]-(A[j]*CN_S[j-1])/alpha[j-1]  
                V[i,jmax] = CN_S[jmax]/alpha[jmax]
                for j in range(jmax-1,jb,-1):
                    V[i,j] = (CN_S[j]-C[j]*V[i,j+1])/alpha[j]
            if i in ipayment[0:2]:
                for j in range(jb+1, jmax+1, 1):
                    V[i,j]=V[i,j]+CPN
                
        jcritreal = S0/dS
        jcrit = int(jcritreal)
        jcritB = int(Barr/dS)+1
        Vcrit = V[0,jcrit]+ (S0 - jcrit * dS) / (dS) * (V[0,jcrit+1] - V[0,jcrit])
        Blambda = (jcritB*dS - Barr)/dS
        output = {'S_steps': jmax, 't_steps': imax, 'CN': Vcrit, 'Barrier Lambda': Blambda}
        cnfdb_result.append(output)

    return cnfdb_result
        

                
                        
                
            
            

In [187]:
cndiscbarr = CNFD(S0,Face,CPN, Barr, T, r, q, sigma, SU, 400, 500, 1, 5460)

In [188]:
df = pd.DataFrame.from_dict(cndiscbarr)
df

,S_steps,t_steps,CN,Barrier Lambda
0,400,5460,945.059622,1.589640e-14
1,401,5460,943.744841,7.200000e-01
2,402,5460,944.487456,4.400000e-01
3,403,5460,943.754340,1.600000e-01
4,404,5460,943.921675,8.800000e-01
...,...,...,...,...
95,495,5460,943.621556,4.000000e-01
96,496,5460,943.033249,1.200000e-01
97,497,5460,943.113450,8.400000e-01
98,498,5460,942.530801,5.600000e-01


In [220]:
df1=list(round(df["Barrier Lambda"],1))


In [243]:
abs(df.iloc[:,3]-0.5).idxmin()

9

In [217]:
df.iloc[df1.index(0.5)]

S_steps            409.000000
t_steps           5460.000000
CN                 944.089858
Barrier Lambda       0.480000
Name: 9, dtype: float64

In [276]:
def CNFD2(S0,Face,CPN, Barr, T, r, q, sigma, SU, jmax, imax):
    cnfdb_result = []
    V = np.zeros([imax+1, jmax+1])
    VT=np.zeros([imax+1, jmax+1])
    A = np.zeros([jmax+1])
    B = np.zeros([jmax+1])
    C = np.zeros([jmax+1])
    D = np.zeros([jmax+1])
    alpha = np.zeros([jmax+1])
    CN_S = np.zeros([jmax+1])
    dt = T / imax
    dS = SU/jmax
    j0=int(S0/dS)
    if (j0*dS < S0): j0 = j0+1
    jb = int(Barr/dS)
    if (jb*dS > Barr): jb = jb-1
    ireview = [90,120,154,181,212,244,273,304,335,365,399,426,455,485,518,546]
    ireview = [round(i/365/T*imax) for i in ireview]
    ipayment=[37,64 ,93,125,157,184,217,247,276,309,338,370,402,429,458,490,521,549]
    ipayment = [round(i/365/T*imax) for i in ipayment]
    i = imax
    for j in range(0, jmax+1):
        if j*dS >= S0:
            VT[i, j] =(Face+CPN)*np.exp(-r*(ipayment[-1]*dt-ireview[-1]*dt))
        else:
            VT[i,j] = (Face*j*dS/S0+CPN)*np.exp(-r*(ipayment[-1]*dt-ireview[-1]*dt))
    for i in range(imax-1, -1, -1):
        if i in ireview:
            A[0] = 0
            B[0] = 1
            C[0] = 0
            D[0] = 0
            for x in (ireview):
                if i <= x:
                    D[0]=D[0]+ CPN*np.exp(-r*(x*dt-i*dt))
            for j in range(0, j0, 1):
                A[j] = 0.25*sigma**2*j**2 - 0.25*(r-q)*j
                B[j] = -1/dt-0.5*r-0.5*sigma**2*j**2
                C[j] = 0.25*sigma**2*j**2 + 0.25*(r-q)*j
                D1 = -VT[i+1,j-1]*(0.25*sigma**2*j**2-0.25*(r-q)*j)
                D2 = -VT[i+1,j]*(1/dt-0.5*r-0.5*sigma**2*j**2)
                D3 = -VT[i+1,j+1]*(0.25*sigma**2*j**2+0.25*(r-q)*j)
                D[j] = D1+D2+D3
            A[j0] = 0
            B[j0] = 1
            C[j0] = 0
            D[j0] = (Face+CPN)*np.exp(-r*(ipayment[ireview.index(i)+2]*dt-i*dt))
            alpha[0] = B[0]
            CN_S[0] = D[0]
            for j in range(1, j0+1, 1):
                alpha[j] = B[j]-(A[j]*C[j-1])/alpha[j-1]
                CN_S[j] = D[j]-(A[j]*CN_S[j-1])/alpha[j-1]  
            VT[i,j0] = CN_S[j0]/alpha[j0]
            for j in range(j0-1,-1,-1):
                VT[i,j] = (CN_S[j]-C[j]*VT[i,j+1])/alpha[j]
                VT[i,j]=VT[i,j]+CPN*np.exp(-r*(ipayment[ireview.index(i)+2]*dt-i*dt))
            for j in range(j0, jmax+1, 1):
                VT[i, j] =(Face+CPN)*np.exp(-r*(ipayment[ireview.index(i)+2]*dt-i*dt))
        else:
            A[0] = 0
            B[0] = 1
            C[0] = 0
            D[0] = 0
            for x in (ireview):
                if x >= i:
                    D[0]=D[0]+ CPN*np.exp(-r*(x*dt-i*dt))
            for j in range(0, jmax, 1):
                A[j] = 0.25*sigma**2*j**2 - 0.25*(r-q)*j
                B[j] = -1/dt-0.5*r-0.5*sigma**2*j**2
                C[j] = 0.25*sigma**2*j**2 + 0.25*(r-q)*j
                D1 = -VT[i+1,j-1]*(0.25*sigma**2*j**2-0.25*(r-q)*j)
                D2 = -VT[i+1,j]*(1/dt-0.5*r-0.5*sigma**2*j**2)
                D3 = -VT[i+1,j+1]*(0.25*sigma**2*j**2+0.25*(r-q)*j)
                D[j] = D1+D2+D3
            A[jmax] = 0
            B[jmax] = 1
            C[jmax] = 0
            for x in range(len(ireview)-1):
                if ireview[x]<i and ireview[x+1]>i:
                    D[jmax] = (Face+CPN)*np.exp(-r*(ipayment[ireview.index(ireview[x+1])+2]*dt-i*dt))
            if i < ireview[0]:
                D[jmax] = (Face+CPN)*np.exp(-r*(ipayment[ireview.index(ireview[0])+2]*dt-i*dt))
            alpha[0] = B[0]
            CN_S[0] = D[0]
            for j in range(1, jmax+1, 1):
                alpha[j] = B[j]-(A[j]*C[j-1])/alpha[j-1]
                CN_S[j] = D[j]-(A[j]*CN_S[j-1])/alpha[j-1]  
            VT[i,jmax] = CN_S[jmax]/alpha[jmax]
            for j in range(jmax-1,-1,-1):
                VT[i,j] = (CN_S[j]-C[j]*VT[i,j+1])/alpha[j]
        if i in ipayment[0:2]:
            VT[i,j]=VT[i,j]+CPN


    i=imax       
    for j in range(0, jmax+1):
        if j*dS >= Barr:
            V[i, j] =(Face+CPN)*np.exp(-r*(ipayment[-1]*dt-ireview[-1]*dt))
        else:
            V[i,j] = (Face*j*dS/S0+CPN)*np.exp(-r*(ipayment[-1]*dt-ireview[-1]*dt)) 
    for i in range(imax-1, -1, -1):
        for j in range(0, jb+1, 1):
            V[i,j]=VT[i,j]
        if i in ireview:
            A[jb] = 0
            B[jb] = 1
            C[jb] = 0
            D[jb] = VT[i,jb]
            for j in range(jb, j0, 1):
                A[j] = 0.25*sigma**2*j**2 - 0.25*(r-q)*j
                B[j] = -1/dt-0.5*r-0.5*sigma**2*j**2
                C[j] = 0.25*sigma**2*j**2 + 0.25*(r-q)*j
                D1 = -V[i+1,j-1]*(0.25*sigma**2*j**2-0.25*(r-q)*j)
                D2 = -V[i+1,j]*(1/dt-0.5*r-0.5*sigma**2*j**2)
                D3 = -V[i+1,j+1]*(0.25*sigma**2*j**2+0.25*(r-q)*j)
                D[j] = D1+D2+D3
            A[j0] = 0
            B[j0] = 1
            C[j0] = 0
            D[j0] = (Face+CPN)*np.exp(-r*(ipayment[ireview.index(i)+2]*dt-i*dt))
            alpha[jb] = B[jb]
            CN_S[jb] = D[jb]
            for j in range(jb, j0+1, 1):
                alpha[j] = B[j]-(A[j]*C[j-1])/alpha[j-1]
                CN_S[j] = D[j]-(A[j]*CN_S[j-1])/alpha[j-1]  
            V[i,j0] = CN_S[j0]/alpha[j0]
            for j in range(j0-1,jb,-1):
                V[i,j] = (CN_S[j]-C[j]*V[i,j+1])/alpha[j]
                V[i,j]=V[i,j]+CPN*np.exp(-r*(ipayment[ireview.index(i)+2]*dt-i*dt))
            for j in range(j0, jmax+1, 1):
                V[i, j] =(Face+CPN)*np.exp(-r*(ipayment[ireview.index(i)+2]*dt-i*dt))
        else:
            A[jb] = 0
            B[jb] = 1
            C[jb] = 0
            D[jb] = VT[i,jb]
            for j in range(jb, jmax, 1):
                A[j] = 0.25*sigma**2*j**2 - 0.25*(r-q)*j
                B[j] = -1/dt-0.5*r-0.5*sigma**2*j**2
                C[j] = 0.25*sigma**2*j**2 + 0.25*(r-q)*j
                D1 = -V[i+1,j-1]*(0.25*sigma**2*j**2-0.25*(r-q)*j)
                D2 = -V[i+1,j]*(1/dt-0.5*r-0.5*sigma**2*j**2)
                D3 = -V[i+1,j+1]*(0.25*sigma**2*j**2+0.25*(r-q)*j)
                D[j] = D1+D2+D3
            A[jmax] = 0
            B[jmax] = 1
            C[jmax] = 0
            for x in range(len(ireview)-1):
                if ireview[x]<i and ireview[x+1]>i:
                    D[jmax] = (Face+CPN)*np.exp(-r*(ipayment[ireview.index(ireview[x+1])+2]*dt-i*dt))
            if i < ireview[0]:
                D[jmax] = (Face+CPN)*np.exp(-r*(ipayment[ireview.index(ireview[0])+2]*dt-i*dt))
            alpha[jb] = B[jb]
            CN_S[jb] = D[jb]
            for j in range(jb+1, jmax+1, 1):
                alpha[j] = B[j]-(A[j]*C[j-1])/alpha[j-1]
                CN_S[j] = D[j]-(A[j]*CN_S[j-1])/alpha[j-1]  
            V[i,jmax] = CN_S[jmax]/alpha[jmax]
            for j in range(jmax-1,jb,-1):
                V[i,j] = (CN_S[j]-C[j]*V[i,j+1])/alpha[j]
            if i in ipayment[0:2]:
                for j in range(jb+1, jmax+1, 1):
                    V[i,j]=V[i,j]+CPN
                
    jcritreal = S0/dS
    jcrit = int(jcritreal)
    jcritB = int(Barr/dS)+1
    Vcrit = V[0,jcrit]+ (S0 - jcrit * dS) / (dS) * (V[0,jcrit+1] - V[0,jcrit])
    Blambda = (jcritB*dS - Barr)/dS
    output = {'S_steps': jmax, 't_steps': imax, 'CN': Vcrit, 'Barrier Lambda': Blambda}
    cnfdb_result.append(output)

    return cnfdb_result
        

In [277]:
cndiscbarr2 = CNFD2(S0,Face,CPN, Barr, T, r, q, sigma, SU, 409,  5460)

In [278]:
df2 = pd.DataFrame.from_dict(cndiscbarr2)
df2

,S_steps,t_steps,CN,Barrier Lambda
0,409,5460,944.089858,0.48
